# Interacting with Campaigns <a class="anchor" id="top"></a>

In this notebook, you will deploy and interact with campaigns in Amazon Personalize.

1. [Introduction](#intro)
1. [Create campaigns](#create)
1. [Interact with campaigns](#interact)
1. [Batch recommendations](#batch)
1. [Wrap up](#wrapup)

## Introduction <a class="anchor" id="intro"></a>
[Back to top](#top)

At this point, you should have several solutions and at least one solution version for each. Once a solution version is created, it is possible to get recommendations from them, and to get a feel for their overall behavior.

This notebook starts off by deploying each of the solution versions from the previous notebook into individual campaigns. Once they are active, there are resources for querying the recommendations, and helper functions to digest the output into something more human-readable. 

As you with your customer on Amazon Personalize, you can modify the helper functions to fit the structure of their data input files to keep the additional rendering working.

To get started, once again, we need to import libraries, load values from previous notebooks, and load the SDK.

In [11]:
import time
from time import sleep
import json
from datetime import datetime
import uuid
import random

import boto3
import pandas as pd

In [12]:
%store -r

In [13]:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

# Establish a connection to Personalize's event streaming
personalize_events = boto3.client(service_name='personalize-events')

## Interact with campaigns <a class="anchor" id="interact"></a>
[Back to top](#top)

Now that all campaigns are deployed and active, we can start to get recommendations via an API call. Each of the campaigns is based on a different recipe, which behave in slightly different ways because they serve different use cases. We will cover each campaign in a different order than used in previous notebooks, in order to deal with the possible complexities in ascending order (i.e. simplest first).

First, let's create a supporting function to help make sense of the results returned by a Personalize campaign. Personalize returns only an `item_id`. This is great for keeping data compact, but it means you need to query a database or lookup table to get a human-readable result for the notebooks. We will create a helper function to return a human-readable result from the LastFM dataset.

Start by loading in the dataset which we can use for our lookup table.

In [14]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(dataset_dir + '/movies.csv', sep=',', usecols=[0,1], encoding='latin-1', dtype={'movieId': "object", 'title': "str"},index_col=0)

# Render some sample data
items_df.head(5)

,title
movieId,
1,Toy Story (1995)
2,Jumanji (1995)
3,Grumpier Old Men (1995)
4,Waiting to Exhale (1995)
5,Father of the Bride Part II (1995)


In [15]:
items_df.dtypes

title    object
dtype: object

By defining the ID column as the index column it is trivial to return an artist by just querying the ID. Movie #589 should be Terminator 2: Judgment Day.

In [16]:
movie_id_example = 589
title = items_df.loc[movie_id_example]['title']
print(title)

Terminator 2: Judgment Day (1991)


That isn't terrible, but it would get messy to repeat this everywhere in our code, so the function below will clean that up.

In [17]:
def get_movie_by_id(movie_id, movie_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return movie_df.loc[int(movie_id)]['title']
    except:
        return "Error obtaining title"

Now let's test a few simple values to check our error catching.

In [18]:
# A known good id (The Princess Bride)
print(get_movie_by_id(movie_id="1197"))
# A bad type of value
print(get_movie_by_id(movie_id="987.9393939"))
# Really bad values
print(get_movie_by_id(movie_id="Steve"))

Princess Bride, The (1987)
Error obtaining title
Error obtaining title


Great! Now we have a way of rendering results. 

### SIMS

SIMS requires just an item as input, and it will return items which users interact with in similar ways to their interaction with the input item. In this particular case the item is a movie. 

The cells below will handle getting recommendations from SIMS and rendering the results. Let's see what the recommendations are for the first item we looked at earlier in this notebook (Terminator 2: Judgment Day).

In [19]:
get_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = sims_campaign_arn,
    itemId = str(589),
)

In [20]:
item_list = get_recommendations_response['itemList']
for item in item_list:
    print(get_movie_by_id(movie_id=item['itemId']))

Jurassic Park (1993)
Braveheart (1995)
Terminator, The (1984)
Fugitive, The (1993)
Speed (1994)
Crimson Tide (1995)
GoldenEye (1995)
Batman (1989)
Clear and Present Danger (1994)
True Lies (1994)
Mask, The (1994)
Die Hard: With a Vengeance (1995)
In the Line of Fire (1993)
Lion King, The (1994)
Ghost (1990)
Forrest Gump (1994)
Apollo 13 (1995)
Cliffhanger (1993)
Star Trek: Generations (1994)
Firm, The (1993)
Die Hard (1988)
Seven (a.k.a. Se7en) (1995)
Indiana Jones and the Last Crusade (1989)
Mission: Impossible (1996)
Mrs. Doubtfire (1993)


Congrats, this is your first list of recommendations! This list is fine, but it would be better to see the recommendations for our sample collection of artists render in a nice dataframe. Again, let's create a helper function to achieve this.

In [21]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df(recommendations_df, movie_ID):
    # Get the movie name
    movie_name = get_movie_by_id(movie_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = sims_campaign_arn,
        itemId = str(movie_ID),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

Now, let's test the helper function with several different movies. Let's sample some data from our dataset to test our SIMS campaign. Grab 5 random movies from our dataframe.

Note: We are going to show similar titles, so you may want to re-run the sample until you recognize some of the movies listed

In [22]:
samples = items_df.sample(5)
samples

,title
movieId,
5365,"Lady and the Duke, The (Anglaise et le duc, L'..."
163981,31 (2016)
138210,13 Hours (2016)
6965,Journeys with George (2002)
91873,Joyful Noise (2012)


In [23]:
sims_recommendations_df = pd.DataFrame()
movies = samples.index.tolist()

for movie in movies:
    sims_recommendations_df = get_new_recommendations_df(sims_recommendations_df, movie)

sims_recommendations_df

,"Lady and the Duke, The (Anglaise et le duc, L') (2001)",31 (2016),13 Hours (2016),Journeys with George (2002),Joyful Noise (2012)
0,"Happy, Texas (1999)",Hatchet III (2013),Hatchet III (2013),"Shawshank Redemption, The (1994)","Shawshank Redemption, The (1994)"
1,Places in the Heart (1984),Stingray Sam (2009),Stingray Sam (2009),Forrest Gump (1994),Forrest Gump (1994)
2,Something Wild (1986),The Boy and the Beast (2015),The Boy and the Beast (2015),Pulp Fiction (1994),Pulp Fiction (1994)
3,"Powerpuff Girls, The (2002)",'71 (2014),'71 (2014),"Silence of the Lambs, The (1991)","Silence of the Lambs, The (1991)"
4,Mrs. Miniver (1942),Devils on the Doorstep (Guizi lai le) (2000),Devils on the Doorstep (Guizi lai le) (2000),Braveheart (1995),Braveheart (1995)
5,Double Happiness (1994),Slow West (2015),Slow West (2015),"Matrix, The (1999)","Matrix, The (1999)"
6,Deathtrap (1982),Hatchet II (2010),31 (2016),Schindler's List (1993),Schindler's List (1993)
7,Empire Records (1995),Ewoks: The Battle for Endor (1985),Hatchet II (2010),Star Wars: Episode IV - A New Hope (1977),Star Wars: Episode IV - A New Hope (1977)
8,84 Charing Cross Road (1987),"Original Kings of Comedy, The (2000)",Ewoks: The Battle for Endor (1985),Jurassic Park (1993),Jurassic Park (1993)
9,Meet John Doe (1941),Frozen (2010),"Original Kings of Comedy, The (2000)",Terminator 2: Judgment Day (1991),Terminator 2: Judgment Day (1991)


You may notice that a lot of the items look the same, hopefully not all of them do (this is more likely with a smaller # of interactions, which will be more common with the movielens small dataset). This shows that the evaluation metrics should not be the only thing you rely on when evaluating your solution version. So when this happens, what can you do to improve the results?

This is a good time to think about the hyperparameters of the Personalize recipes. The SIMS recipe has a `popularity_discount_factor` hyperparameter (see [documentation](https://docs.aws.amazon.com/personalize/latest/dg/native-recipe-sims.html)). Leveraging this hyperparameter allows you to control the nuance you see in the results. This parameter and its behavior will be unique to every dataset you encounter, and depends on the goals of the business. You can iterate on the value of this hyperparameter until you are satisfied with the results, or you can start by leveraging Personalize's hyperparameter optimization (HPO) feature. For more information on hyperparameters and HPO tuning, see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/customizing-solution-config-hpo.html).

### User Personalization

HRNN is one of the more advanced algorithms provided by Amazon Personalize. It supports personalization of the items for a specific user based on their past behavior and can intake real time events in order to alter recommendations for a user without retraining. 

Since HRNN relies on having a sampling of users, let's load the data we need for that and select 3 random users. Since Movielens does not include user data, we will select 3 random numbers from the range of user id's in the dataset.

In [24]:
if not USE_FULL_MOVIELENS:
    users = random.sample(range(1, 600), 3)
else:
    users = random.sample(range(1, 162000), 3)
users

[28, 281, 503]

Now we render the recommendations for our 3 random users from above. After that, we will explore real-time interactions before moving on to Personalized Ranking.

Again, we create a helper function to render the results in a nice dataframe.

#### API call results

In [25]:
# Update DF rendering
pd.set_option('display.max_rows', 30)

def get_new_recommendations_df_users(recommendations_df, user_id):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [user_id])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [26]:
recommendations_df_users = pd.DataFrame()
#users = users_df.sample(3).index.tolist()

for user in users:
    recommendations_df_users = get_new_recommendations_df_users(recommendations_df_users, user)

recommendations_df_users

,28,281,503
0,Apocalypse Now (1979),Best in Show (2000),"Truman Show, The (1998)"
1,Enemy at the Gates (2001),Little Miss Sunshine (2006),"Wolf of Wall Street, The (2013)"
2,"Pianist, The (2002)",What Women Want (2000),Pitch Perfect (2012)
3,"Pan's Labyrinth (Laberinto del fauno, El) (2006)",Bridget Jones's Diary (2001),"Help, The (2011)"
4,Amadeus (1984),Pulp Fiction (1994),"Hunt, The (Jagten) (2012)"
5,Big Fish (2003),Chicago (2002),"Green Mile, The (1999)"
6,"Killing Fields, The (1984)",Snatch (2000),Star Trek Into Darkness (2013)
7,Inglourious Basterds (2009),"Rocky Horror Picture Show, The (1975)",Black Swan (2010)
8,One Flew Over the Cuckoo's Nest (1975),"Others, The (2001)",50/50 (2011)
9,Finding Neverland (2004),Catch Me If You Can (2002),Warrior (2011)


Here we clearly see that the recommendations for each user are different. If you were to need a cache for these results, you could start by running the API calls through all your users and store the results, or you could use a batch export, which will be covered later in this notebook.

### Static and Dynamic Filters

Lets interact with the static filters we created in the previous notebook, and utilize dynamic filters in realtime.

A few common use cases for dynamic filters in Video On Demand are:

Categorical filters based on Item Metadata (that arent range based) - Often your item metadata will have information about the title such as Genre, Keyword, Year, Director, Actor etc. Filtering on these can provide recommendations within that data, such as action movies, Steven Spielberg movies, Movies from 1995 etc.

Events - you may want to filter out certain events and provide results based on those events, such as moving a title from a "suggestions to watch" recommendation to a "watch again" recommendations.

Now lets apply item filters to see recommendations for one of these users within each decade of our static filters.


In [27]:
def get_new_recommendations_df_by_static_filter(recommendations_df, user_id, filter_arn):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = filter_arn
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    #print(recommendation_list)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_name])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

In [28]:
def get_new_recommendations_df_by_dynamicfilter(recommendations_df, user_id, genre_filter_arn, filter_values):
    # Get the movie name
    #movie_name = get_movie_by_id(artist_ID)
    # Get the recommendations
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
        filterArn = genre_filter_arn,
        filterValues = { "GENRE": "\"" + filter_values + "\""}
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        movie = get_movie_by_id(item['itemId'])
        recommendation_list.append(movie)
    filter_name = filter_arn.split('/')[1]
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [filter_values])
    # Add this dataframe to the old one
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

You can see the recommendations for movies within a given decade. Within a VOD application you could create Shelves (also known as rails or carosels) easily by using these filters. Depending on the information you have about your items, You could also filter on additional information such as keyword, year/decade etc.

In [29]:
recommendations_df_decade_shelves = pd.DataFrame()
for filter_arn in decade_filter_arns:
    recommendations_df_decade_shelves = get_new_recommendations_df_by_static_filter(recommendations_df_decade_shelves, user, filter_arn)

recommendations_df_decade_shelves

,1950s,1960s,1970s,1980s,1990s,2000s,2010s
0,12 Angry Men (1957),Cool Hand Luke (1967),Star Wars: Episode IV - A New Hope (1977),"Karate Kid, The (1984)","Truman Show, The (1998)",Burn After Reading (2008),"Wolf of Wall Street, The (2013)"
1,"Streetcar Named Desire, A (1951)",2001: A Space Odyssey (1968),"Godfather, The (1972)",Star Wars: Episode VI - Return of the Jedi (1983),"Green Mile, The (1999)",Dr. Horrible's Sing-Along Blog (2008),Pitch Perfect (2012)
2,On the Waterfront (1954),Oliver! (1968),"Clockwork Orange, A (1971)",NausicaÃ¤ of the Valley of the Wind (Kaze no t...,Private Parts (1997),Pay It Forward (2000),"Help, The (2011)"
3,"20,000 Leagues Under the Sea (1954)",Dr. Strangelove or: How I Learned to Stop Worr...,"Godfather: Part II, The (1974)","Fish Called Wanda, A (1988)",Trainspotting (1996),Marley & Me (2008),"Hunt, The (Jagten) (2012)"
4,Vertigo (1958),Bonnie and Clyde (1967),Little Big Man (1970),Superman II (1980),Cruel Intentions (1999),Gosford Park (2001),Star Trek Into Darkness (2013)
5,Rashomon (RashÃ´mon) (1950),Breakfast at Tiffany's (1961),Solaris (Solyaris) (1972),"Good Morning, Vietnam (1987)",William Shakespeare's Romeo + Juliet (1996),Gran Torino (2008),Black Swan (2010)
6,"Trouble with Harry, The (1955)","Nutty Professor, The (1963)","Rocky Horror Picture Show, The (1975)",Ferris Bueller's Day Off (1986),Lone Star (1996),Avatar (2009),50/50 (2011)
7,Forbidden Planet (1956),"Graduate, The (1967)",One Flew Over the Cuckoo's Nest (1975),This Is Spinal Tap (1984),Notting Hill (1999),Mystic River (2003),Warrior (2011)
8,Anatomy of a Murder (1959),"Hard Day's Night, A (1964)",Dog Day Afternoon (1975),Terms of Endearment (1983),October Sky (1999),Man on Fire (2004),Snowpiercer (2013)
9,Dial M for Murder (1954),Planet of the Apes (1968),"Andromeda Strain, The (1971)",Jean de Florette (1986),"Fifth Element, The (1997)","Devil Wears Prada, The (2006)",Hot Tub Time Machine (2010)


In [30]:
# Create a dataframe for the items by reading in the correct source CSV
items_meta_df = pd.read_csv(data_dir + '/item-meta.csv', sep=',', index_col=0)

# Render some sample data
items_meta_df.head(10)

,GENRE,YEAR,CREATION_TIMESTAMP
ITEM_ID,,,
1,Adventure|Animation|Children|Comedy|Fantasy,1995,0
2,Adventure|Children|Fantasy,1995,0
3,Comedy|Romance,1995,0
4,Comedy|Drama|Romance,1995,0
5,Comedy,1995,0
6,Action|Crime|Thriller,1995,0
7,Comedy|Romance,1995,0
8,Adventure|Children,1995,0
9,Action,1995,0


Now what we want to do is determine the genres to filter on, for that we need a list of all genres. First we will get all the unique values of the column GENRE, then split strings on | if they exist, everyone will then get added to a long list which will be converted to a set for efficiency. That set will then be made into a list so that it can be iterated, and we can then use the get recommendatioins API.

In [31]:
unique_genre_field_values = items_meta_df['GENRE'].unique()

genre_val_list = []

def process_for_bar_char(val, val_list):
    if '|' in val:
        values = val.split('|')
        for item in values:
            val_list.append(item)
    elif '(' in val:
        pass
    else:
        val_list.append(val)
    return val_list
    

for val in unique_genre_field_values:
    genre_val_list = process_for_bar_char(val, genre_val_list)

genres_to_filter = list(set(genre_val_list))

In [32]:
genres_to_filter

['Children',
 'Romance',
 'Western',
 'Film-Noir',
 'Drama',
 'Documentary',
 'Crime',
 'Thriller',
 'Adventure',
 'Fantasy',
 'Horror',
 'Mystery',
 'Sci-Fi',
 'Musical',
 'Comedy',
 'Animation',
 'War',
 'IMAX',
 'Action']

In [33]:
# Iterate through Genres
recommendations_df_genre_shelves = pd.DataFrame()
for genre in genres_to_filter:
    recommendations_df_genre_shelves = get_new_recommendations_df_by_dynamicfilter(recommendations_df_genre_shelves, user, genre_filter_arn , genre)
    
recommendations_df_genre_shelves

,Children,Romance,Western,Film-Noir,Drama,Documentary,Crime,Thriller,Adventure,Fantasy,Horror,Mystery,Sci-Fi,Musical,Comedy,Animation,War,IMAX,Action
0,Hugo (2011),Midnight in Paris (2011),Lone Star (1996),L.A. Confidential (1997),"Truman Show, The (1998)",Exit Through the Gift Shop (2010),"Wolf of Wall Street, The (2013)",Black Swan (2010),Star Trek Into Darkness (2013),Midnight in Paris (2011),Let Me In (2010),Let Me In (2010),"Truman Show, The (1998)",Pitch Perfect (2012),"Truman Show, The (1998)",Family Guy Presents: It's a Trap (2010),Incendies (2010),Star Trek Into Darkness (2013),Star Trek Into Darkness (2013)
1,Megamind (2010),Moonrise Kingdom (2012),True Grit (2010),Drive (2011),"Wolf of Wall Street, The (2013)",Catfish (2010),"Green Mile, The (1999)",Captain Phillips (2013),Captain Phillips (2013),Man of Steel (2013),"Cabin in the Woods, The (2012)",Incendies (2010),Star Trek Into Darkness (2013),Holy Motors (2012),"Wolf of Wall Street, The (2013)",Frozen (2013),Embassy (2013),Captain Phillips (2013),Snowpiercer (2013)
2,"Karate Kid, The (2010)",William Shakespeare's Romeo + Juliet (1996),Cowboys & Aliens (2011),Rubber (2010),"Help, The (2011)",Kevin Hart: Laugh at My Pain (2011),Trainspotting (1996),Inception (2010),"Amazing Spider-Man, The (2012)","Wolverine, The (2013)",Resident Evil: Retribution (2012),Holy Motors (2012),Snowpiercer (2013),"History of Future Folk, The (2012)",Pitch Perfect (2012),Ice Age 4: Continental Drift (2012),Life Is Beautiful (La Vita Ã¨ bella) (1997),"Amazing Spider-Man, The (2012)","Amazing Spider-Man, The (2012)"
3,Despicable Me (2010),Notting Hill (1999),"Lone Ranger, The (2013)",Tinker Tailor Soldier Spy (2011),"Hunt, The (Jagten) (2012)",Inside Job (2010),Inception (2010),"Cabin in the Woods, The (2012)",Man of Steel (2013),Holy Motors (2012),Prometheus (2012),Inception (2010),Hot Tub Time Machine (2010),Frozen (2013),50/50 (2011),Dr. Seuss' The Lorax (2012),Courage Under Fire (1996),Gravity (2013),Gravity (2013)
4,Despicable Me 2 (2013),Frozen (2013),Django Unchained (2012),Lost Highway (1997),"Green Mile, The (1999)",Jackass 3D (2010),Mud (2012),Super 8 (2011),"Wolverine, The (2013)",Ted (2012),"Thing, The (2011)",Lone Star (1996),"Amazing Spider-Man, The (2012)",Rock of Ages (2012),Hot Tub Time Machine (2010),"Adventures of Tintin, The (2011)",Transformers: Dark of the Moon (2011),Man of Steel (2013),Man of Steel (2013)
5,Space Jam (1996),"MisÃ©rables, Les (2012)",Wild Wild West (1999),Bullet to the Head (2012),Black Swan (2010),Pearl Jam Twenty (2011),Horrible Bosses (2011),The Hunger Games (2012),"Day of the Doctor, The (2013)",Hereafter (2010),Resident Evil: Afterlife (2010),I'll Follow You Down (2013),Melancholia (2011),"MisÃ©rables, Les (2012)","Descendants, The (2011)",Wreck-It Ralph (2012),Captain America: The First Avenger (2011),Cloud Atlas (2012),"Wolverine, The (2013)"
6,"Iron Giant, The (1999)","Crazy, Stupid, Love. (2011)",Rubber (2010),2 Days in the Valley (1996),50/50 (2011),Restrepo (2010),We're the Millers (2013),Prisoners (2013),"History of Future Folk, The (2012)",Harry Potter and the Deathly Hallows: Part 2 (...,"Crazies, The (2010)",Super 8 (2011),Gravity (2013),Dr. Seuss' The Lorax (2012),Intouchables (2011),Into the Forest of Fireflies' Light (2011),Generation War (2013),Inception (2010),Inception (2010)
7,Tangled (2010),Her (2013),"Warrior's Way, The (2010)",Dark City (1998),Warrior (2011),Jackass 3.5 (2011),Burn After Reading (2008),Rise of the Planet of the Apes (2011),Harry Potter and the Deathly Hallows: Part 2 (...,Frozen (2013),Tucker & Dale vs Evil (2010),Harry Potter and the Deathly Hallows: Part 2 (...,Man of Steel (2013),Dr. Horrible's Sing-Along Blog (2008),Louis C.K.: Live at the Beacon Theater (2011),It's Such a Beautiful Day (2012),X-Men: First Class (2011),Super 8 (2011),Harry Potter and the Deathly Hallows: Part 2 (...
8,Shrek Forever After (a.k.a. Shrek: The Final C...,Shakespeare in Love (1998),Jonah Hex (2010),Hoodlum (1997),Snowpiercer (

The next topic is real-time events. Personalize has the ability to listen to events from your application in order to update the recommendations shown to the user. This is especially useful in media workloads, like video-on-demand, where a customer's intent may differ based on if they are watching with their children or on their own.

Additionally the events that are recorded via this system are stored until a delete call from you is issued, and they are used as historical data alongside the other interaction data you provided when you train your next models.

#### Real time events

Start by creating an event tracker that is attached to the campaign.

In [34]:
response = personalize.create_event_tracker(
    name='MovieTracker',
    datasetGroupArn=dataset_group_arn
)
print(response['eventTrackerArn'])
print(response['trackingId'])
TRACKING_ID = response['trackingId']
event_tracker_arn = response['eventTrackerArn']

arn:aws:personalize:us-east-1:832194813872:event-tracker/dff308a5
6c4ead98-cec9-47bf-9963-6dcf9489012d


We will create some code that simulates a user interacting with a particular item. After running this code, you will get recommendations that differ from the results above.

We start by creating some methods for the simulation of real time events.

In [35]:
session_dict = {}

def send_movie_click(USER_ID, ITEM_ID, EVENT_TYPE):
    """
    Simulates a click as an envent
    to send an event to Amazon Personalize's Event Tracker
    """
    # Configure Session
    try:
        session_ID = session_dict[str(USER_ID)]
    except:
        session_dict[str(USER_ID)] = str(uuid.uuid1())
        session_ID = session_dict[str(USER_ID)]
        
    # Configure Properties:
    event = {
    "itemId": str(ITEM_ID),
    }
    event_json = json.dumps(event)
        
    # Make Call
    
    personalize_events.put_events(
    trackingId = TRACKING_ID,
    userId= str(USER_ID),
    sessionId = session_ID,
    eventList = [{
        'sentAt': int(time.time()),
        'eventType': str(EVENT_TYPE),
        'properties': event_json
        }]
    )

def get_new_recommendations_df_users_real_time(recommendations_df, user_id, item_id, event_type):
    # Get the artist name (header of column)
    movie_name = get_movie_by_id(item_id)
    # Interact with different movies
    print('sending event ' + event_type + ' for ' + get_movie_by_id(item_id))
    send_movie_click(USER_ID=user_id, ITEM_ID=item_id, EVENT_TYPE=event_type)
    # Get the recommendations (note you should have a base recommendation DF created before)
    get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )
    # Build a new dataframe of recommendations
    item_list = get_recommendations_response['itemList']
    recommendation_list = []
    for item in item_list:
        artist = get_movie_by_id(item['itemId'])
        recommendation_list.append(artist)
    new_rec_DF = pd.DataFrame(recommendation_list, columns = [movie_name])
    # Add this dataframe to the old one
    #recommendations_df = recommendations_df.join(new_rec_DF)
    recommendations_df = pd.concat([recommendations_df, new_rec_DF], axis=1)
    return recommendations_df

At this point, we haven't generated any real-time events yet; we have only set up the code. To compare the recommendations before and after the real-time events, let's pick one user and generate the original recommendations for them.

In [36]:
# First pick a user
user_id = user

# Get recommendations for the user
get_recommendations_response = personalize_runtime.get_recommendations(
        campaignArn = userpersonalization_campaign_arn,
        userId = str(user_id),
    )

# Build a new dataframe for the recommendations
item_list = get_recommendations_response['itemList']
recommendation_list = []
for item in item_list:
    artist = get_movie_by_id(item['itemId'])
    recommendation_list.append(artist)
user_recommendations_df = pd.DataFrame(recommendation_list, columns = [user_id])
user_recommendations_df

,503
0,"Truman Show, The (1998)"
1,"Wolf of Wall Street, The (2013)"
2,Pitch Perfect (2012)
3,"Help, The (2011)"
4,"Hunt, The (Jagten) (2012)"
5,"Green Mile, The (1999)"
6,Star Trek Into Darkness (2013)
7,Black Swan (2010)
8,50/50 (2011)
9,Warrior (2011)


Ok, so now we have a list of recommendations for this user before we have applied any real-time events. Now let's pick 3 random artists which we will simulate our user interacting with, and then see how this changes the recommendations.

In [37]:
# Next generate 3 random movies
movies = items_df.sample(3).index.tolist()

In [38]:
# Note this will take about 15 seconds to complete due to the sleeps
for movie in movies:
    user_recommendations_df = get_new_recommendations_df_users_real_time(user_recommendations_df, user_id, movie,'click')
    time.sleep(5)

sending event click for He Never Died (2015)
sending event click for Catfish (2010)
sending event click for Ali (2001)


Now we can look at how the click events changed the recommendations.

In [39]:
user_recommendations_df

,503,He Never Died (2015),Catfish (2010),Ali (2001)
0,"Truman Show, The (1998)","Truman Show, The (1998)",Captain Phillips (2013),Catfish (2010)
1,"Wolf of Wall Street, The (2013)","Wolf of Wall Street, The (2013)",Black Swan (2010),Inside Job (2010)
2,Pitch Perfect (2012),Pitch Perfect (2012),"Green Mile, The (1999)",Into the Abyss (2011)
3,"Help, The (2011)","Help, The (2011)",Notting Hill (1999),Wolf Children (Okami kodomo no ame to yuki) (2...
4,"Hunt, The (Jagten) (2012)","Hunt, The (Jagten) (2012)",William Shakespeare's Romeo + Juliet (1996),Death Note Rewrite: Genshisuru Kami (2007)
5,"Green Mile, The (1999)","Green Mile, The (1999)","Girl with the Dragon Tattoo, The (2011)",Everest (1998)
6,Star Trek Into Darkness (2013),Star Trek Into Darkness (2013),"Truman Show, The (1998)",Citizen Kane (1941)
7,Black Swan (2010),Black Swan (2010),Gravity (2013),"Secret of Kells, The (2009)"
8,50/50 (2011),50/50 (2011),Donnie Brasco (1997),Who Killed the Electric Car? (2006)
9,Warrior (2011),Warrior (2011),Midnight in Paris (2011),Searching for Sugar Man (2012)


In the cell above, the first column after the index is the user's default recommendations from User Personalization, and each column after that has a header of the artist that they interacted with via a real time event, and the recommendations after this event occurred. 

The behavior may not shift very much; this is due to the relatively limited nature of this dataset and effect of a few random clicks. If you wanted to better understand this, try simulating clicking more movies, and you should see a more pronounced impact.

Now lets look at the event filters, which allow you to filter items based on the interaction data. For this dataset, it could be click or watch based on the data we imported, but could be based on whatever interaction schema you design (click, rate, like, watch, purchase etc.) For VOD shelves you could move a title from "Top picks for you" to a "Watch again", the watch again recommendations will be based on the users current interactions, but only recommend titles that have already been watched.


In [40]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_static_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Almost Famous (2000),Waking Life (2001)
1,Memento (2000),Spirited Away (Sen to Chihiro no kamikakushi) ...
2,Ghost World (2001),Billy Elliot (2000)
3,American Beauty (1999),Men of Honor (2000)
4,Fight Club (1999),Pay It Forward (2000)
5,One Flew Over the Cuckoo's Nest (1975),Pollock (2000)
6,Gummo (1997),8 Mile (2002)
7,Hugo (2011),"Magdalene Sisters, The (2002)"
8,"Shawshank Redemption, The (1994)",Gosford Park (2001)
9,"Scanner Darkly, A (2006)",Dancer in the Dark (2000)


now lets send a watch event in for 4 unwatched recommendations, which would simulate watching 4 movies. In a VOD application, you may choose to send in an event after they have watched a significant amount (over 75%) of a piece of content. Sending at 100% complete could miss people that stop short of the credits.

In [41]:
 # Get the recommendations
top_unwatched_recommendations_response = personalize_runtime.get_recommendations(
    campaignArn = userpersonalization_campaign_arn,
    userId = str(user_id),
    filterArn = filter_arn,
    numResults=4)
item_list = top_unwatched_recommendations_response['itemList']
for item in item_list:
    print('sending event watch for ' + get_movie_by_id(item['itemId']))
    send_movie_click(USER_ID=user_id, ITEM_ID=item['itemId'], EVENT_TYPE='watch')
    time.sleep(10)

sending event watch for Waking Life (2001)
sending event watch for Spirited Away (Sen to Chihiro no kamikakushi) (2001)
sending event watch for Billy Elliot (2000)
sending event watch for Men of Honor (2000)


Now we can look at the event filters to see the updated watched and unwatched recommendations 

In [42]:
recommendations_df_events = pd.DataFrame()
for filter_arn in interaction_filter_arns:
    recommendations_df_events = get_new_recommendations_df_by_static_filter(recommendations_df_events, user, filter_arn)
    
recommendations_df_events

,watched,unwatched
0,Billy Elliot (2000),"Truman Show, The (1998)"
1,Pulp Fiction (1994),"Wolf of Wall Street, The (2013)"
2,Almost Famous (2000),Pitch Perfect (2012)
3,Spirited Away (Sen to Chihiro no kamikakushi) ...,"Help, The (2011)"
4,"Shawshank Redemption, The (1994)","Hunt, The (Jagten) (2012)"
5,Waking Life (2001),Star Trek Into Darkness (2013)
6,Fight Club (1999),Black Swan (2010)
7,Lost in Translation (2003),50/50 (2011)
8,One Flew Over the Cuckoo's Nest (1975),Warrior (2011)
9,Forrest Gump (1994),Snowpiercer (2013)


### Personalized Ranking

The core use case for personalized ranking is to take a collection of items and to render them in priority or probable order of interest for a user. For a VOD application you want dynamically render a personalized shelf/rail/carousel based on some information (director, location, superhero franchise, movie time period etc). This may not be information that you have in your metadata, so a item metadata filter will not work, howeverr you may have this information within you system to generate the item list. 

To demonstrate this, we will use the same user from before and a random collection of items.

In [43]:
rerank_user = user
rerank_items = items_df.sample(25).index.tolist()

Now build a nice dataframe that shows the input data.

In [44]:
rerank_list = []
for item in rerank_items:
    movie = get_movie_by_id(item)
    rerank_list.append(movie)
rerank_df = pd.DataFrame(rerank_list, columns = ['Un-Ranked'])
rerank_df

,Un-Ranked
0,Rory Scovel Tries Stand-Up for the First Time ...
1,Predator 2 (1990)
2,Grumpier Old Men (1995)
3,Best of the Best (1989)
4,La cravate (1957)
5,"Secret of Roan Inish, The (1994)"
6,"Journey of Natty Gann, The (1985)"
7,Sunflower (Xiang ri kui) (2005)
8,Hart's War (2002)
9,"Dark Half, The (1993)"


Then make the personalized ranking API call.

In [45]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

Now add the reranked items as a second column to the original dataframe, for a side-by-side comparison.

In [46]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    movie = get_movie_by_id(item['itemId'])
    ranked_list.append(movie)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,Un-Ranked,Re-Ranked
0,Rory Scovel Tries Stand-Up for the First Time ...,Charlotte Gray (2001)
1,Predator 2 (1990),Little Miss Sunshine (2006)
2,Grumpier Old Men (1995),Hart's War (2002)
3,Best of the Best (1989),Sunflower (Xiang ri kui) (2005)
4,La cravate (1957),Carol (2015)
5,"Secret of Roan Inish, The (1994)",The Good Dinosaur (2015)
6,"Journey of Natty Gann, The (1985)","Reader, The (2008)"
7,Sunflower (Xiang ri kui) (2005),"Secret of Roan Inish, The (1994)"
8,Hart's War (2002),Bless the Child (2000)
9,"Dark Half, The (1993)",Swimfan (2002)


You can see above how each entry was re-ordered based on the model's understanding of the user. This is a popular task when you have a collection of items to surface a user, a list of promotions for example.

## Batch recommendations <a class="anchor" id="batch"></a>
[Back to top](#top)

There are many cases where you may want to have a larger dataset of exported recommendations. Recently, Amazon Personalize launched batch recommendations as a way to export a collection of recommendations to S3. In this example, we will walk through how to do this for the HRNN solution. For more information about batch recommendations, please see the [documentation](https://docs.aws.amazon.com/personalize/latest/dg/getting-recommendations.html#recommendations-batch). This feature applies to all recipes, but the output format will vary.

A simple implementation looks like this:

```python
import boto3

personalize_rec = boto3.client(service_name='personalize')

personalize_rec.create_batch_inference_job (
    solutionVersionArn = "Solution version ARN",
    jobName = "Batch job name",
    roleArn = "IAM role ARN",
    jobInput = 
       {"s3DataSource": {"path": S3 input path}},
    jobOutput = 
       {"s3DataDestination": {"path":S3 output path"}}
)
```

The SDK import, the solution version arn, and role arns have all been determined. This just leaves an input, an output, and a job name to be defined.

Starting with the input for HRNN, it looks like:


```JSON
{"userId": "4638"}
{"userId": "663"}
{"userId": "3384"}
```

This should yield an output that looks like this:

```JSON
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```

The output is a JSON Lines file. It consists of individual JSON objects, one per line. So we will need to put in more work later to digest the results in this format.

### Building the input file

When you are using the batch feature, you specify the users that you'd like to receive recommendations for when the job has completed. The cell below will again select a few random users and will then build the file and save it to disk. From there, you will upload it to S3 to use in the API call later.

In [37]:
# We will use the same users from before
users
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir + "/" + json_input_filename, 'w') as json_input:
    for user_id in users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [38]:
# Showcase the input file:
!cat $data_dir"/"$json_input_filename

{"userId": "381"}
{"userId": "573"}
{"userId": "431"}


Upload the file to S3 and save the path as a variable for later.

In [39]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket_name).Object(json_input_filename).upload_file(data_dir+"/"+json_input_filename)
s3_input_path = "s3://" + bucket_name + "/" + json_input_filename
print(s3_input_path)

s3://832194813872-us-east-1-personalizepocvod/json_input.json


Batch recommendations read the input from the file we've uploaded to S3. Similarly, batch recommendations will save the output to file in S3. So we define the output path where the results should be saved.

In [40]:
# Define the output path
s3_output_path = "s3://" + bucket_name + "/"
print(s3_output_path)

s3://832194813872-us-east-1-personalizepocvod/


Now just make the call to kick off the batch export process.

In [41]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = userpersonalization_solution_version_arn,
    jobName = "VOD-POC-Batch-Inference-Job-UserPersonalization_" + str(round(time.time()*1000)),
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

Run the while loop below to track the status of the batch recommendation call. This can take around 30 minutes to complete, because Personalize needs to stand up the infrastructure to perform the task. We are testing the feature with a dataset of only 3 users, which is not an efficient use of this mechanism. Normally, you would only use this feature for bulk processing, in which case the efficiencies will become clear.

In [42]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 6*60*60 # 6 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  12:28:45 AM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

In [43]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket_name, export_name, data_dir+"/"+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open(data_dir+"/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            movie = get_movie_by_id(item)
            recommendation_list.append(movie)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])
        try:
            line = json.loads(json_file.readline())
        except:
            line = None
bulk_recommendations_df

,User: 381,User: 573,User: 431
0,"Pursuit of Happyness, The (2006)",Up (2009),Phenomenon (1996)
1,Into the Wild (2007),"Dark Knight, The (2008)",Sliding Doors (1998)
2,Slumdog Millionaire (2008),District 9 (2009),"Crying Game, The (1992)"
3,Scarface (1983),Raiders of the Lost Ark (Indiana Jones and the...,"Time to Kill, A (1996)"
4,Walk the Line (2005),"Godfather: Part II, The (1974)",Sense and Sensibility (1995)
5,"Notebook, The (2004)",Inglourious Basterds (2009),Leaving Las Vegas (1995)
6,Atonement (2007),"Pan's Labyrinth (Laberinto del fauno, El) (2006)",Conspiracy Theory (1997)
7,Harry Potter and the Deathly Hallows: Part 1 (...,WALLÂ·E (2008),Jerry Maguire (1996)
8,X-Men (2000),"Lord of the Rings: The Return of the King, The...","Postman, The (Postino, Il) (1994)"
9,Top Gun (1986),"Godfather, The (1972)","Bodyguard, The (1992)"


## Wrap up <a class="anchor" id="wrapup"></a>
[Back to top](#top)

With that you now have a fully working collection of models to tackle various recommendation and personalization scenarios, as well as the skills to manipulate customer data to better integrate with the service, and a knowledge of how to do all this over APIs and by leveraging open source data science tools.

Use these notebooks as a guide to getting started with your customers for POCs. As you find missing components, or discover new approaches, cut a pull request and provide any additional helpful components that may be missing from this collection.

You'll want to make sure that you clean up all of the resources deployed during this POC. We have provided a separate notebook which shows you how to identify and delete the resources in `06_Clean_Up_Resources.ipynb`.

In [47]:
%store event_tracker_arn
%store TRACKING_ID
%store batchInferenceJobArn


Stored 'event_tracker_arn' (str)
Stored 'TRACKING_ID' (str)
